In [332]:
import json
import math

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.neighbors import KNeighborsClassifier
from tqdm import tqdm
from datetime import datetime
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler


In [333]:
def ret(n, n_1):
    return (n_1 - n) / n


PREDICTIONS = 700
# prices every 15 seconds
# df = pd.read_csv("prices.csv")
f = open("BTC_USDT-3m.json")
df = pd.DataFrame.from_records(json.load(f), columns=['timestamp', 'open', 'high', 'low', 'close', 'vol'])
# df.columns = ['price']
df['ret_c'] = (df['close'] - df['close'].shift(1)) / df['close'].shift(1)
df['ret_o'] = (df['open'] - df['open'].shift(1)) / df['open'].shift(1)

df['ret_h'] = (df['high'] - df['high'].shift(1)) / df['high'].shift(1)
df['ret_l'] = (df['low'] - df['low'].shift(1)) / df['low'].shift(1)

df['mid'] = (df['low'] + df['high'])/2

df['ret'] = (df['open'].shift(-1) - df['open']) / df['open']
df['close_n'] = df['close'].shift(-1)
mean_ret = 0.0001
df['ret'] = df['ret'].apply(lambda x: np.sign(x))
print(len(df[df['ret'] == 0]))
print(len(df[df['ret'] == -1]))
print(len(df[df['ret'] == 1]))

df['co_d'] = df['close'] - df['open']


def perc_dif(actual, prediction):
    return np.abs(prediction - actual) / prediction

2
2597
2648


In [334]:
df

,timestamp,open,high,low,close,vol,ret_c,ret_o,ret_h,ret_l,mid,ret,close_n,co_d
0,1661904000000,19813.03,19838.84,19811.14,19819.50,582.13461,NaN,NaN,NaN,NaN,19824.990,1.0,19863.03,6.47
1,1661904180000,19820.97,19876.77,19797.94,19863.03,605.82699,0.002196,0.000401,0.001912,-0.000666,19837.355,1.0,19904.99,42.06
2,1661904360000,19865.04,19910.78,19861.16,19904.99,578.33857,0.002112,0.002223,0.001711,0.003193,19885.970,1.0,19873.44,39.95
3,1661904540000,19905.80,19910.18,19864.75,19873.44,607.35949,-0.001585,0.002052,-0.000030,0.000181,19887.465,-1.0,19883.66,-32.36
4,1661904720000,19873.44,19892.60,19863.52,19883.66,354.87011,0.000514,-0.001626,-0.000883,-0.000062,19878.060,1.0,19891.34,10.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5243,1662847740000,21695.72,21747.90,21675.87,21725.07,914.47974,0.001353,-0.000084,0.001325,0.000476,21711.885,1.0,21723.08,29.35
5244,1662847920000,21727.16,21735.53,21690.00,21723.08,710.86427,-0.000092,0.001449,-0.000569,0.000652,21712.765,-1.0,21689.83,-4.08
5245,1662848100000,21722.02,21729.23,21663.22,21689.83,844.59582,-0.001531,-0.000237,-0.000290,-0.001235,21696.225,-1.0,21658.57,-32.19
5246,1662848280000,21689.83,21697.99,21653.75,21658.57,746.62426,-0.001441,-0.001482,-0.001438,-0.000437,21675.870,-1.0,21688.59,-31.26


In [339]:
hit_rates = []
normalized = "non_norm_ohlc_3m_det"
for K in np.arange(13, 14):
    for NDTR in tqdm((np.arange(K, 500, 10))):
        nans = 0
        actuals = []
        predictions = []
        hits = 0
        for i in np.arange(NDTR, len(df)):
            if len(predictions) < PREDICTIONS:
                small_df = df.iloc[i: i + NDTR + 1].copy(deep=True)[
                    ['ret_o','ret_c', 'ret_h','ret_l','ret']].reset_index(
                    drop=True)

                if len(small_df) > 0:
                    x_train = small_df.iloc[0:NDTR][['ret_c', 'ret_h','ret_l','ret_o']].values
                    y_train = small_df['ret'].iloc[0:NDTR].values
                    st_x= RobustScaler()
                    x_train= st_x.fit_transform(x_train)
                    knn2 = KNeighborsClassifier(n_neighbors=K,weights="distance")
                    knn2.fit(x_train, y_train)
                    if len(small_df) > NDTR:
                        x_test = [small_df.iloc[NDTR][['ret_o','ret_h','ret_l', 'ret_c']].values]
                        x_test= st_x.transform(x_test)

                        prediction = knn2.predict(x_test)
                        if math.isnan(prediction):
                            nans += 1
                            prediction = 0
                        else:
                            actual = small_df['ret'].iloc[NDTR]
                            prediction = prediction[0]
                            if actual == prediction:
                                hits += 1
                            actuals.append(actual)
                            predictions.append(prediction)
            else:
                i = len(df)
        if len(actuals) > 0:
            small_df = df.iloc[NDTR:NDTR + 50]
            hit_rates.append([hits / len(actuals), NDTR, len(actuals), K])
            # print('1', len(list(filter(lambda x: x == 1, predictions))))
            # print('-1', len(list(filter(lambda x: x == -1, predictions))))
            # print('0', len(list(filter(lambda x: x == 0, predictions))))

        # fig = make_subplots(specs=[[{"secondary_y": True}]])
        # fig.add_trace(
        #     go.Scatter(x=small_df['timestamp'].apply(lambda x: datetime.fromtimestamp(x / 1000)), y=actuals,
        #                name="Retorno reales", mode='lines'))
        # fig.add_trace(
        #     go.Scatter(x=small_df['timestamp'].apply(lambda x: datetime.fromtimestamp(x / 1000)), y=predictions,
        #                name="Retorno predichos", mode='lines'), secondary_y=True)
        # title = "PNN K=" + str(NDTR) + normalized + "|HitRate=" + "{:.2%}".format(hits / len(actuals))
        # fig.update_layout(title=title, xaxis_title="Real", yaxis_title="Prediction")
        # # fig.show()
        # fig.write_image("knn/prediction_vs_real_K" + str(NDTR) + "_std" + normalized + ".png")

    # small_df = df.iloc[NDTR: 50+NDTR]
    # fig = make_subplots()
    # fig.add_trace(
    #     go.Scatter(x=small_df['timestamp'].apply(lambda x: datetime.fromtimestamp(x / 1000)), y=small_df['ret'],
    #                 name="Retornos", mode='lines+markers'))
    # fig.add_trace(
    #     go.Scatter(x=small_df['timestamp'].apply(lambda x: datetime.fromtimestamp(x / 1000)), y=predictions[NDTR:],
    #                name="Retornos predichos", mode='lines+markers'))
    # fig.update_layout(title="Predicciones con NDTR=" + str(NDTR))
    # fig.show()

In [340]:
hit_df = pd.DataFrame(hit_rates, columns=['hit_rate', 'NDTR', 'predictions', 'K'])
print(hit_df)
hit_df.to_csv("knn/knn_hit_rates" + normalized + "_withK2.csv", header=None, index=None)

   hit_rate  NDTR  predictions   K
0     0.474   158          500  13


In [341]:
# error_df = pd.read_csv("ret_h/oc/errors.csv")
# error_df.columns = ['error', 'std', 'NDTR']
# 55.57
# other_errors = pd.read_csv("ret_h/oc/errors.csv")
# other_errors.columns = ['error', 'std', 'NDTR']

# error_df = pd.concat([other_errors, error_df])

fig = make_subplots()

# for NDTR in error_df['NDTR'].unique():
#     small_df = error_df[error_df['NDTR'] == NDTR]
#     actual_hdf= hit_df[hit_df['NDTR'] == NDTR]
#     fig.add_trace(go.Scatter(x=actual_hdf['std'], y=actual_hdf['hit_rate'].apply(lambda x: 1 - x), name="NDTR=" + str(NDTR), mode="markers+lines"))

for K in np.arange(2,15):
    small_df = hit_df[hit_df['K'] ==K]
    fig.add_trace(
        go.Scatter(x=small_df['NDTR'], y=small_df['hit_rate'], mode="markers+lines", name="K="+str(K)))

fig.update_layout(yaxis_tickformat=".2%", title="Hit rate of prediction for a given K", xaxis_title="NDTR",
                  yaxis_title="Hit rate", xaxis_tickformat="none")
fig.write_image("knn/hitrate_withK.png")

fig.show()